In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
import random
from datetime import datetime
import re
import pickle
import json

start = time.time()

category = ['268'] # 네이버 카테고리 입력

dt_index = pd.date_range(start='20210806', end='20210831') # 기간 입력
dt_list = dt_index.strftime("%Y%m%d").tolist()

for idx, ca in enumerate(category):
    print(ca)
    # url 수집
    url_press_list = []
    for date in dt_list:
        
        page_num = 1
        last = False
        
        while last == False:
            raw = requests.get("https://news.naver.com/main/list.naver?mode=LS2D&sid2="+ca+"&mid=shm&sid1=101&date="+date+"&page="+str(page_num),
                                    headers={"User-Agent":"Mozilla/5.0"})
            html = BeautifulSoup(raw.text, 'html.parser')

            page = html.find('div', {'class':'paging'})
            try:
                page_a_list = page.findAll('a')

                try:
                    if '다음' in str(page_a_list[-1]):
                        page_num += 10
                    else:
                        last = True
                        try:
                            page_num = int(page_a_list[-1].text)
                        except:
                            page_num = page_num
                except:
                    last = True
                    page_num = 1
            
            except:
                page_num += 10
                
        time.sleep(random.randrange(3,10))
                

        for i in range(1, page_num+1):  
            raw = requests.get("https://news.naver.com/main/list.naver?mode=LS2D&sid2="+ca+"&mid=shm&sid1=101&date="+date+"&page="+str(i),
                            headers={"User-Agent":"Mozilla/5.0"})
            html = BeautifulSoup(raw.text, 'html.parser')                

            news_container = html.select("div.list_body.newsflash_body li dl")

            for news in news_container:
                try:
                    url = news.select_one("dt:nth-child(2) a")['href']
                except:
                    url = news.select_one("dt:nth-child(1) a")['href']
                
                try:
                    press = news.select_one("span.writing").text
                except:
                    press = "NA"
                
                url_press_list.append([url, press])

    time.sleep(random.randrange(10,15))
    
    print(len(url_press_list))
    
    # 내용 수집
    content_list = []
    for url_idx, url_press in enumerate(url_press_list):
        try:
            raw = requests.get(url_press[0], headers={"User-Agent":"Mozilla/5.0"})
            html = BeautifulSoup(raw.text, 'html.parser')    

            ID = "100" + ('%02d' % (idx + 3)) + "_" + str(url_idx + 1) # 아이디

            # 날짜
            try:
                date = html.select_one("div.sponsor > span:nth-child(1)").text
                date = date.replace(".", "-")[:10]
                date = datetime.strptime(date, '%Y-%m-%d').date()

            except:
                date = html.select_one("div.sponsor > span:nth-child(2)").text
                date = date.replace(".", "-")[:10]
                date = datetime.strptime(date, '%Y-%m-%d').date()        

            Date = date

            Title = html.select_one('h3#articleTitle').text # 제목
            Content = html.select_one('div#articleBodyContents').text # 내용
            Content_len = float(len(Content)) # 내용의 길이

            # 댓글
            Comment = []

            oid = url_press[0].split("oid=")[1].split("&")[0]
            aid = url_press[0].split("aid=")[1]
            c_page = 1

            header = {
                "User-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/65.0.3325.181 Safari/537.36",
                "referer": url_press[0],
            }

            while True:
                c_url = "https://apis.naver.com/commentBox/cbox/web_neo_list_jsonp.json?ticket=news&templateId=default_society&pool=cbox5&_callback=jQuery1707138182064460843_1523512042464&lang=ko&country=&objectId=news" + oid + "%2C" + aid + "&categoryId=&pageSize=20&indexSize=10&groupId=&listType=OBJECT&pageType=more&page=" + str(
                    c_page) + "&refresh=false&sort=FAVORITE"

                c_raw = requests.get(c_url, headers=header)
                c_html = BeautifulSoup(c_raw.content, "html.parser")
                total_comm = str(c_html).split('comment":')[1].split(",")[0]

                c_content = re.findall('"contents":([^\*]*),"userIdNo"', str(c_html))
                c_like = re.findall('"sympathyCount":([^\*]*),"antipathyCount"', str(c_html))
                c_dislike = re.findall('"antipathyCount":([^\*]*),"hideReplyButton"', str(c_html))

                for i in range(len(c_content)):
                    Comment.append([c_content[i], float(c_like[i]), float(c_dislike[i])])

                if int(total_comm) <= ((c_page) * 20):
                    break
                else:
                    c_page += 1

            Comment_cnt = float(len(Comment))

            if len(Comment) == 0:
                Comment = "NA"

            # 공감
            Sentiment = {"좋아요" : 0, "훈훈해요" : 0, "슬퍼요" : 0, "화나요" : 0, "후속기사 원해요" : 0}

            s_url = "https://news.like.naver.com/v1/search/contents?suppress_response_codes=true&callback=jQuery11240039925253521598814_1631253269105&q=NEWS%5Bne_" + oid + "_" + aid + "%5D%7CNEWS_SUMMARY%5B087_0000859977%5D%7CJOURNALIST%5B75301(period)%5D%7CNEWS_MAIN%5Bne_087_0000859977%5D&isDuplication=false&_=1631253269107"
            s_raw = requests.get(s_url, headers=header)

            idx1 = s_raw.text.find('(')
            idx2 = s_raw.text.find(')')

            s_jsonData = s_raw.text[idx1+1:idx2]
            s_jsonData = json.loads(s_jsonData)

            for i in range(len(s_jsonData['contents'][0]['reactions'])):
                Sentiment[s_jsonData['contents'][0]['reactions'][i]['reactionTypeCode']['description']] = s_jsonData['contents'][0]['reactions'][i]['count']

            Link = html.select_one("div.sponsor a.btn_artialoriginal")['href'] # 링크
            Press = url_press[1].replace(" ", "") # 언론사
            
            content_list.append([ID, Date, Title, Content, Content_len, Comment, Comment_cnt, Sentiment, Link, Press])

            if url_idx % 300 == 0:
                time.sleep(random.randrange(30,50))
                print(ID)
                
                
        except:
            continue
            
            
    # 파일 저장   
    df = pd.DataFrame(content_list, columns =['ID', 'Date', 'Title', 'Content', 'Content_len', 'Comment', 'Comment_cnt', 'Sentiment', 'Link', 'Press'])

    print(df)
    with open('파일명을 입력하세요', 'wb') as f: # 저장 파일명 입력
        pickle.dump(df, f)

    f.close()
    print("time :", time.time() - start)

268
12514
10003_301
10003_601
10003_1501
10003_2101
10003_2401
10003_2701
10003_3301
10003_3901
10003_4201
10003_4501
10003_5101
10003_5401
10003_5701
10003_6001
10003_6301
10003_6601
10003_6901
10003_7201
10003_8101
10003_8701
10003_9301
10003_9601
10003_10201
10003_10801
10003_11101
10003_12001
10003_12301
               ID        Date                                           Title  \
0        10003_12  2021-05-01        美 전문가들, 바이든 행정부 새 대북정책에 '회의적'…"이미 실패한 길"   
1        10003_15  2021-05-01             '트럼프도 오바마도 아닌' 바이든式 대북정책..北 호응에 달렸다   
2        10003_16  2021-05-01               北 선전매체 "대북 전단 대신 '탈북자 쓰레기'날려라" 비판   
3        10003_20  2021-05-01             "조정되고 실용적인 접근"…바이든표 대북 정책 어떤 내용 담길까   
4        10003_21  2021-05-01  美 연습기 조립서 KF-21까지… 100년 만에 편 ‘자주국방 날개’ [S 스토리]   
...           ...         ...                                             ...   
9467  10003_12508  2021-08-31                 한미, 美서 북핵 후속 협의…성김 "北 답변 듣길 기대"   
9468  10003_12510  2021-08-31             